In [1]:
import pandas as pd
import os
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import torch
from collections import defaultdict
import json
from tqdm import tqdm

from ultralytics import YOLO

In [2]:
# Dataset dir
dataset_dir = '/app/tennis_data/stroke_recognition_dataset/master/v1.0'
match_info_json_path = os.path.join(dataset_dir, "match_info.json")

match_info_df = pd.read_json(match_info_json_path)
dataset_df = pd.read_csv(os.path.join(dataset_dir, "clean_dataset_info.csv"))

In [3]:
dataset_df.head()

,filename,original_filename,top_cls,btm_cls,match_id,fps,ballhit_match_timestamp,ballhit_match_frame_pos,surface
0,t_serve_00001_b_other_00001_m000.mp4,m000_p000_bh00_serve_top_Novak_Djokovic.mp4,serve,other,0,25.0,00:27:31.896,41297,hardcourt
1,t_serve_00002_b_other_00002_m000.mp4,m000_p000_bh01_serve_top_Novak_Djokovic.mp4,serve,other,0,25.0,00:27:41.370,41534,hardcourt
2,t_other_00001_b_forehand_00001_m000.mp4,m000_p000_bh02_forehand_bottom_Andy_Murray.mp4,other,forehand,0,25.0,00:27:42.113,41552,hardcourt
3,t_serve_00003_b_other_00003_m000.mp4,m000_p001_bh00_serve_top_Novak_Djokovic.mp4,serve,other,0,25.0,00:28:02.918,42072,hardcourt
4,t_other_00002_b_backhand_00001_m000.mp4,m000_p001_bh01_backhand_bottom_Andy_Murray.mp4,other,backhand,0,25.0,00:28:03.661,42091,hardcourt


In [4]:
match_info_df

,match_id,filename,match_start_top,match_start_bottom,fps,first_point_start_frame,surface,tournament,year,round
0,0,000_Novak_Djokovic_v_Andy_Murray_Australian_Op...,Novak Djokovic,Andy Murray,25.000000,41250,hardcourt,Australian Open,2016,F
1,1,001_Novak_Djokovic_v_Rafael_Nadal_Australian_O...,Novak Djokovic,Rafael Nadal,25.000000,18000,hardcourt,Australian Open,2012,F
2,2,002_Novak_Djokovic_v_Roger_Federer_US_Open_201...,Roger Federer,Novak Djokovic,29.970030,500,hardcourt,US Open,2015,F
3,3,003_Novak_Djokovic_v_Roger_Federer_Wimbledon_2...,Roger Federer,Novak Djokovic,25.000000,350,grass,Wimbledon,2019,F
4,4,004_Rafael_Nadal_v_Nick_Kyrgios_Wimbledon_2019...,Rafael Nadal,Nick Kyrgios,25.000000,0,grass,Wimbledon,2019,R2
5,5,005_Roger_Federer_v_Rafael_Nadal_Wimbledon_201...,Roger Federer,Rafael Nadal,25.000000,450,grass,Wimbledon,2019,SF
6,7,007_Stan_Wawrinka_v_Novak_Djokovic_US_Open_201...,Novak Djokovic,Stan Wawrinka,23.976024,0,hardcourt,US Open,2016,F
7,8,008_Novak_Djokovic_v_Rafael_Nadal_Australian_O...,Rafael Nadal,Novak Djokovic,29.941000,7440,hardcourt,Australian Open,2019,F
8,22,022_Novak_Djokovic_v_Rafael_Nadal_Wimbledon_20...,Novak Djokovic,Rafael Nadal,25.000000,0,grass,Wimbledon,2018,SF
9,23,023_Alexander_Zverev_v_Dominic_Thiem_US_Open_2...,Dominic Thiem,Alexander Zverev,29.970030,0,hardcourt,US Open,2020,F


In [5]:
subset_df = dataset_df[dataset_df['match_id'].isin([0, 1, 2, 3, 4, 5])]
# subset_df = dataset_df[dataset_df['match_id'].isin([7, 8, 22, 23, 24])]
# subset_df = dataset_df[dataset_df['match_id'].isin([25, 100, 101, 102, 103, 104, 105])]
# subset_df = dataset_df[dataset_df['match_id'].isin([106, 107, 108, 109, 110])]
# subset_df = dataset_df[dataset_df['match_id'].isin([111, 112, 113, 114])]

In [6]:
subset_df.shape

(2871, 9)

In [7]:
subset_df.head()

,filename,original_filename,top_cls,btm_cls,match_id,fps,ballhit_match_timestamp,ballhit_match_frame_pos,surface
0,t_serve_00001_b_other_00001_m000.mp4,m000_p000_bh00_serve_top_Novak_Djokovic.mp4,serve,other,0,25.0,00:27:31.896,41297,hardcourt
1,t_serve_00002_b_other_00002_m000.mp4,m000_p000_bh01_serve_top_Novak_Djokovic.mp4,serve,other,0,25.0,00:27:41.370,41534,hardcourt
2,t_other_00001_b_forehand_00001_m000.mp4,m000_p000_bh02_forehand_bottom_Andy_Murray.mp4,other,forehand,0,25.0,00:27:42.113,41552,hardcourt
3,t_serve_00003_b_other_00003_m000.mp4,m000_p001_bh00_serve_top_Novak_Djokovic.mp4,serve,other,0,25.0,00:28:02.918,42072,hardcourt
4,t_other_00002_b_backhand_00001_m000.mp4,m000_p001_bh01_backhand_bottom_Andy_Murray.mp4,other,backhand,0,25.0,00:28:03.661,42091,hardcourt


In [8]:
def extract_frames(video_path):
    cap = cv.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    cap.release()
    return np.array(frames)

def resample_frames(frames, num_output_frames=20):
    total_frames = len(frames)
    indices = np.linspace(0, total_frames - 1, num=num_output_frames, dtype=int)
    resampled_frames = frames[indices]
    return resampled_frames


def write_video(output_path, frames, fps=20):
    height, width, layers = frames[0].shape
    size = (width, height)
    
    fourcc = cv.VideoWriter_fourcc(*'mp4v')  # You can change 'mp4v' to other formats like 'XVID'
    out = cv.VideoWriter(output_path, fourcc, fps, size, isColor=True)
    
    for frame in frames:
        out.write(frame)

    out.release()

In [9]:
for i in tqdm(range(subset_df.shape[0]), desc="Processing videos", unit="video"):
    row = subset_df.iloc[i]
    frames = extract_frames(os.path.join(dataset_dir, "vids", row['original_filename']))
    if 'serve' in (row['top_cls'], row['btm_cls']):
        if row['fps'] <= 24:
            frames = frames[:20]
        if 24 < row['fps'] < 26:
            frames = frames[:21]
        else:
            frames = frames[:len(frames) - 5]
    resampled_frames = resample_frames(frames)
    # assert len(resampled_frames) == 20
    output_path = os.path.join(dataset_dir, "final_dataset_20_frames", row['filename'])
    # print(output_path)
    write_video(output_path, resampled_frames)

Processing videos: 100% 2871/2871 [11:23<00:00,  4.20video/s]
